In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb

from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
data = pd.read_csv("../input/twilio-stock-price-twlo20162021/TWLO_stock.csv")
data

In [ ]:
data['date']
# 2016-06-23 --- 2021-06-10

In [ ]:
n=len(data)
print(n)

In [ ]:
t=0
for item in data['date']:
    if item[0:4]=='2021':
        t+=1
print(t) 

In [ ]:
testx=data[n-110:]
testx

In [ ]:
def dummy_date(df):
    df["year"] = df["date"].apply(lambda x: x.split("-")[0])
    df["month"] = df["date"].apply(lambda x: x.split("-")[1])
    df["day"] = df["date"].apply(lambda x: x.split("-")[2])
    df.drop("date",inplace=True,axis=1)
    return df

def LabelEncord_categorical(df):
    categorical_params = ["year","month","day"]
    for params in categorical_params:
        le = LabelEncoder()
        df[params] = le.fit_transform(df[params])
    return df

def dummies(df):
    categorical_params = ["year","month","day"]
    for params in categorical_params:
        dummies =  pd.get_dummies(df[params])
        df = pd.concat([df, dummies],axis=1)
    return df

def pre_processing(df):
    df = dummy_date(df)
    df = LabelEncord_categorical(df)
#    df = dummies(df)
    return df

data2 = pre_processing(data)

In [ ]:
train=data2[0:n-110]
test=data2[n-110:]

In [ ]:
train

In [ ]:
test

In [ ]:
y_train = train["4. close"].values
X_train = train.drop("4. close",axis=1).values
y_test = test["4. close"].values
#X_test = test.drop("4. close",axis=1).values

In [ ]:
# To predict 4.close values in the condition of 2.high and 3.low values unkown
x_test = test.drop("4. close",axis=1)
x_test[["2. high","3. low"]]=None
X_test = x_test.values
x_test

### GridSearch for searching best hyperparameter   

In [ ]:
gbm = xgb.XGBRegressor()
reg_cv = GridSearchCV(gbm, {"colsample_bytree":[1.0],"min_child_weight":[1.0,1.2],
                            'max_depth': [3,4,6], 'n_estimators': [500,1000]}, verbose=1)
reg_cv.fit(X_train,y_train)

In [ ]:
reg_cv.best_params_

### Train data using XGBRegressor with best parameter

In [ ]:
gbm = xgb.XGBRegressor(**reg_cv.best_params_)
gbm.fit(X_train,y_train)

### Predict

In [ ]:
predictions = gbm.predict(X_test)

In [ ]:
gbm.score(X_test,y_test)

In [ ]:
gbm.score(X_train,y_train)

In [ ]:
testx['pred_close']=predictions
testx2=testx[['date','4. close','pred_close']]
testx2

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=testx2['date'],y=testx2['4. close'],name="real close values"),secondary_y=False,)
fig.add_trace(go.Scatter(x=testx2['date'],y=testx2['pred_close'],name="predicted close values"),secondary_y=False,)

fig.update_layout(autosize=False,width=700,height=500,title_text="TWILIO predicted Close values compared to real Close values")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Value",secondary_y=False)
fig.show()